In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader 
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np

In [ ]:
NUM_TRAIN = 
NUM_TOTALIMAGES = 
DataSetFolder = 
BATCHSIZE =  #cifar10 set to 64
R_Mean = 
G_Mean = 
B_Mean = 
R_Std = 
G_Std = 
B_Std = 



In [ ]:
## This cell is drafted after Assignment2 Pytorch - but those images were already square!!!!

## to normalize by subtracting the mean RGB vlaue and dividing by the standard deviations of each RGB value
## the Mean and STD are from CIFAR10 dataset
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((R_Mean, G_Mean, B_Mean), (R_std, G_Std, B_Std))])

poster_train = dset.POSTERS(DataSetFolder, train=True, download =True,
                             transform=transform)

loader_train = DataLoader(poster_train, batch_size=BATCHSIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

poster_val = dset.POSTERS(DataSetFolder, train=True, download=True, transform=transform)

loader_val = DataLoader(poster_val, batch_size=BATCHSIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_TOTALIMAGES)))

poster_test = dset.POSTERS(DataSetFolder, train=False, download=True, transform=transform)

loader_test = DataLoader(poster_test, batch_size=BATCHSIZE)

In [ ]:
USE_GPU = True
dtype = torch.float32 

if USE_GPU and torch.cuda.is_available(): 
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

In [ ]:
def flatten(x):
    N = x.shape[0] #read in N, C, H, W
    return x.view(N, -1) # "flatten" the C * H * W values into a sing le vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2) 
    print('Before flattening: ', x) 
    print('After flattening: ', flatten(x))

test_flatten()

In [ ]:
class Flatten(nn.Module):
    def forward(self, x): 
        return flatten(x)

In [ ]:
def check_accuracy(loader, model): 
    if loader.dataset.train:
        print('Checking accuracy on validation set') 
    else:
        print('Checking accuracy on test set')

    num_correct = 0
    num_samples = 0
    model.eval() # set model to evaluation mode 
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype) # move to device, e.
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples , 100 * acc))

In [ ]:
def train_part(model, optimizer, epochs=1): 
    """
     - model: A PyTorch Module giving the model to train.
     - optimizer: An Optimizer object we will use to train the model
     - epochs: (Optional) A Python integer giving the number of epochs to train for
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device) # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train() # put model to training mode
            x = x.to(device=device, dtype=dtype) # move to device, e.
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            loss = F.cross_entropy(scores, y)
            
            # Zero out all of the gradients for the variables which th
            # will update.
            
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the
            # respect to each  parameter of the model.
            loss.backward()
            
            # Actually update the parameters of the model using the gr
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %s' % (e))
                print('Iteration %d, loss = %.4f' % (t, loss.item())) 
                check_accuracy(loader_val, model)
                print()

In [ ]:
channel_1 = 32
channel_2 = 16
channel_3 = 32
channel_4 = 16
learning_rate = 1e-2
in_channel = 3
num_classes = 10
num_features = 3*32*32

model = nn.Sequential(
#Layer 1
    nn.Conv2d(in_channel, channel_1, 5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=1, padding=1),
    nn.BatchNorm2d(channel_1),
#Layer2
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(channel_2),
    #Layer2.2 ====> added in as an experiment
    nn.Conv2d(channel_2, channel_3, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=1, padding=1),
    nn.BatchNorm2d(channel_3),
#Layer3
    nn.Conv2d(channel_3, channel_4, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=1, padding=1),
    nn.BatchNorm2d(channel_4),
#Layer4
    Flatten(),
    nn.Linear((channel_4*32*32), num_classes)
)

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

train_function(model, optimizer, epochs=10)